Only keep (not ablate) the query activations (query vector, the output of query weights times inputs) of certain positions. Keep all the key activations; the query positions that were kept will automatically attend to relevant key positions by matrix multiplication.

# Setup
(No need to change anything)

## import libs

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-wv_f53k8
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-wv_f53k8
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 174209ea708fe3838ccf08b70f2f4f28e7397cb4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
    

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [ ]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [ ]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [ ]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [ ]:
model = HookedTransformer.from_pretrained(
    "gpt2-medium",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-medium into HookedTransformer


## Import functions from repo

In [ ]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9100, done.
remote: Counting objects: 100% (1814/1814), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 9100 (delta 1609), reused 1602 (delta 1523), pack-reused 7286
Receiving objects: 100% (9100/9100), 155.60 MiB | 14.75 MiB/s, done.
Resolving deltas: 100% (5502/5502), done.


In [ ]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [ ]:
import ioi_circuit_extraction as ioi_circuit_extraction

# test prompts

In [ ]:
modeltest = HookedTransformer.from_pretrained("gpt2-medium")

Loaded pretrained model gpt2-medium into HookedTransformer


In [ ]:
example_prompt = "2 4 6 8 10 12"
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 0        Logit: 16.38 Prob: 44.89% Token: | 14|

Top 0th token. Logit: 16.38 Prob: 44.89% Token: | 14|
Top 1th token. Logit: 15.45 Prob: 17.68% Token: | 13|
Top 2th token. Logit: 14.83 Prob:  9.55% Token: | 16|
Top 3th token. Logit: 14.32 Prob:  5.73% Token: | 15|
Top 4th token. Logit: 13.99 Prob:  4.10% Token: |
|
Top 5th token. Logit: 13.13 Prob:  1.74% Token: | 12|
Top 6th token. Logit: 13.11 Prob:  1.70% Token: | 20|
Top 7th token. Logit: 13.09 Prob:  1.68% Token: | 18|
Top 8th token. Logit: 12.44 Prob:  0.88% Token: | 19|
Top 9th token. Logit: 12.22 Prob:  0.70% Token: | 1|


Ranks of the answer tokens: [(' 14', 0)]

In [ ]:
example_prompt = "2 4 6 8 10 12 14 16"
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12', ' 14', ' 16']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 27       Logit: 10.00 Prob:  0.13% Token: | 14|

Top 0th token. Logit: 15.87 Prob: 47.20% Token: | 18|
Top 1th token. Logit: 14.79 Prob: 16.02% Token: |
|
Top 2th token. Logit: 14.38 Prob: 10.61% Token: | 17|
Top 3th token. Logit: 13.73 Prob:  5.57% Token: | 20|
Top 4th token. Logit: 13.17 Prob:  3.15% Token: | 16|
Top 5th token. Logit: 12.57 Prob:  1.73% Token: | 19|
Top 6th token. Logit: 11.29 Prob:  0.48% Token: | 22|
Top 7th token. Logit: 11.22 Prob:  0.45% Token: | 24|
Top 8th token. Logit: 11.21 Prob:  0.45% Token: | 1|
Top 9th token. Logit: 11.19 Prob:  0.44% Token: | 32|


Ranks of the answer tokens: [(' 14', 27)]

In [ ]:
example_prompt = "3 5 7 9 11 13"
example_answer = " 15"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '3', ' 5', ' 7', ' 9', ' 11', ' 13']
Tokenized answer: [' 15']


Performance on answer token:
Rank: 0        Logit: 16.39 Prob: 17.90% Token: | 15|

Top 0th token. Logit: 16.39 Prob: 17.90% Token: | 15|
Top 1th token. Logit: 16.36 Prob: 17.34% Token: | 14|
Top 2th token. Logit: 16.19 Prob: 14.60% Token: |
|
Top 3th token. Logit: 15.31 Prob:  6.03% Token: | 13|
Top 4th token. Logit: 15.29 Prob:  5.94% Token: | 16|
Top 5th token. Logit: 14.69 Prob:  3.26% Token: | 17|
Top 6th token. Logit: 14.26 Prob:  2.11% Token: | 19|
Top 7th token. Logit: 14.17 Prob:  1.94% Token: | 20|
Top 8th token. Logit: 13.95 Prob:  1.55% Token: | 12|
Top 9th token. Logit: 13.82 Prob:  1.37% Token: | 18|


Ranks of the answer tokens: [(' 15', 0)]

In [ ]:
example_prompt = "3 5 7 9 11 11"
example_answer = " 15"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '3', ' 5', ' 7', ' 9', ' 11', ' 11']
Tokenized answer: [' 15']


Performance on answer token:
Rank: 15       Logit: 12.90 Prob:  0.81% Token: | 15|

Top 0th token. Logit: 16.88 Prob: 43.29% Token: |
|
Top 1th token. Logit: 13.94 Prob:  2.30% Token: | 1|
Top 2th token. Logit: 13.94 Prob:  2.29% Token: | 11|
Top 3th token. Logit: 13.73 Prob:  1.86% Token: | 3|
Top 4th token. Logit: 13.69 Prob:  1.79% Token: | 12|
Top 5th token. Logit: 13.63 Prob:  1.68% Token: | 7|
Top 6th token. Logit: 13.55 Prob:  1.56% Token: | 9|
Top 7th token. Logit: 13.50 Prob:  1.48% Token: | 14|
Top 8th token. Logit: 13.42 Prob:  1.36% Token: | 2|
Top 9th token. Logit: 13.41 Prob:  1.35% Token: | 0|


Ranks of the answer tokens: [(' 15', 15)]

In [ ]:
example_prompt = "2 4 8 16 32"
example_answer = " 64"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 8', ' 16', ' 32']
Tokenized answer: [' 64']


Performance on answer token:
Rank: 0        Logit: 15.46 Prob: 17.84% Token: | 64|

Top 0th token. Logit: 15.46 Prob: 17.84% Token: | 64|
Top 1th token. Logit: 14.86 Prob:  9.78% Token: | 36|
Top 2th token. Logit: 14.63 Prob:  7.81% Token: | 48|
Top 3th token. Logit: 14.38 Prob:  6.05% Token: | 40|
Top 4th token. Logit: 14.27 Prob:  5.42% Token: | 32|
Top 5th token. Logit: 14.18 Prob:  4.94% Token: |
|
Top 6th token. Logit: 13.81 Prob:  3.44% Token: | 44|
Top 7th token. Logit: 13.60 Prob:  2.78% Token: | 4|
Top 8th token. Logit: 13.50 Prob:  2.51% Token: | 34|
Top 9th token. Logit: 13.44 Prob:  2.37% Token: | 42|


Ranks of the answer tokens: [(' 64', 0)]

# Generate dataset with multiple prompts

In [ ]:
class Dataset:
    def __init__(self, prompts, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [ ]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'S5': str(i+4),
            'S6': str(i+5),
            'corr': str(i+6),
            'incorr': str(i+5),
            'text': f"{i} {i+1} {i+2} {i+3} {i+4} {i+5}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 11)
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [ ]:
def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'S5': str(i+4),
            'S6': str(i+4),
            'corr': str(i+5),
            'incorr': str(i+6),
            'text': f"{i} {i+1} {i+2} {i+3} {i+4} {i+4}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(1, 11)
dataset_2 = Dataset(prompts_list_2, model.tokenizer, S1_is_first=True)

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablation Expm Functions

In [ ]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

We can also prevent redundant computation of the full circuit score by storing it and just passing it in to the function.

In [ ]:
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

tensor(6.3172, device='cuda:0')

In [ ]:
def mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=True):
    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    # ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model_abl = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model_abl(dataset.toks)

    # orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

In [ ]:
lst = [(0, 0)]

CIRCUIT = {
    "number mover": lst,
    "number mover 5": lst,
    "number mover 4": lst,
    "number mover 3": lst,
    "number mover 2": lst,
    "number mover 1": lst,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
model_abl = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model_abl(dataset.toks)
logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)

tensor(0.3795, device='cuda:0')

# Ablate the model and compare with original

In [ ]:
lst = [(layer, head) for layer in range(24) for head in range(16)]

CIRCUIT = {
    "number mover": lst,
    "number mover 5": lst,
    "number mover 4": lst,
    "number mover 3": lst,
    "number mover 2": lst,
    "number mover 1": lst,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

100.0

In [ ]:
def circuit_from_headsList(headsList):
    CIRCUIT = {
    "number mover": headsList,
    "number mover 5": headsList,
    "number mover 4": headsList,
    "number mover 3": headsList,
    "number mover 2": headsList,
    "number mover 1": headsList,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        "number mover 5": "S5",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }
    return CIRCUIT, SEQ_POS_TO_KEEP

In [ ]:
CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList([(0,0)])
mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

6.0066447257995605

In [ ]:
# Start with full circuit
curr_circuit = [(layer, head) for layer in range(24) for head in range(16)]
threshold = 3  # This is T, a %. if performance is less than T%, allow its removal

for layer in range(23, -1, -1):  # go thru all heads in a layer first
    for head in range(16):
        # Copying the curr_circuit so we can iterate over one and modify the other
        copy_circuit = curr_circuit.copy()

        # Temporarily removing the current tuple from the copied circuit
        copy_circuit.remove((layer, head))

        CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList(copy_circuit)
        new_score = mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

        print((layer,head), new_score)
        # If the result is less than the threshold, remove the tuple from the original list
        if (100 - new_score) < threshold:
            curr_circuit.remove((layer, head))

            print("Removed:", (layer, head))
            print(new_score)
            print("\n")

(23, 0) 100.07615661621094
Removed: (23, 0)
100.07615661621094


(23, 1) 100.14395904541016
Removed: (23, 1)
100.14395904541016


(23, 2) 100.10014343261719
Removed: (23, 2)
100.10014343261719


(23, 3) 99.94171905517578
Removed: (23, 3)
99.94171905517578


(23, 4) 100.39082336425781
Removed: (23, 4)
100.39082336425781


(23, 5) 99.92205810546875
Removed: (23, 5)
99.92205810546875


(23, 6) 99.66873168945312
Removed: (23, 6)
99.66873168945312


(23, 7) 99.7561264038086
Removed: (23, 7)
99.7561264038086


(23, 8) 98.87322998046875
Removed: (23, 8)
98.87322998046875


(23, 9) 98.9052505493164
Removed: (23, 9)
98.9052505493164


(23, 10) 99.04598999023438
Removed: (23, 10)
99.04598999023438


(23, 11) 99.04353332519531
Removed: (23, 11)
99.04353332519531


(23, 12) 99.1275405883789
Removed: (23, 12)
99.1275405883789


(23, 13) 99.15987396240234
Removed: (23, 13)
99.15987396240234


(23, 14) 99.16203308105469
Removed: (23, 14)
99.16203308105469


(23, 15) 99.1469955444336
Removed: (23, 15)

In [ ]:
curr_circuit

[(0, 9),
 (1, 2),
 (1, 11),
 (1, 14),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 7),
 (2, 9),
 (2, 10),
 (2, 14),
 (3, 0),
 (3, 3),
 (3, 4),
 (3, 9),
 (3, 13),
 (3, 15),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 13),
 (5, 5),
 (5, 11),
 (5, 12),
 (5, 13),
 (6, 12),
 (6, 15),
 (7, 2),
 (8, 10),
 (9, 5),
 (10, 1),
 (10, 4),
 (10, 14),
 (11, 1),
 (11, 3),
 (11, 4),
 (11, 5),
 (11, 6),
 (12, 15),
 (13, 13),
 (14, 5),
 (14, 14),
 (15, 7),
 (15, 10),
 (17, 12),
 (19, 1)]

In [ ]:
len(curr_circuit)

47

384 total

## comapre to 246

In [ ]:
circ_123 = [(0, 9), (1, 2), (1, 11), (1, 14), (2, 3), (2, 4), (2, 5), (2, 7), (2, 9), (2, 10), (2, 14), (3, 0), (3, 3), (3, 4), (3, 9), (3, 13), (3, 15), (4, 6), (4, 7), (4, 8), (4, 9), (4, 13), (5, 5), (5, 11), (5, 12), (5, 13), (6, 12), (6, 15), (7, 2), (8, 10), (9, 5), (10, 1), (10, 4), (10, 14), (11, 1), (11, 3), (11, 4), (11, 5), (11, 6), (12, 15), (13, 13), (14, 5), (14, 14), (15, 7), (15, 10), (17, 12), (19, 1)]
len(circ_123)

47

In [ ]:
circ_246 =  [(0, 2), (0, 3), (0, 4), (0, 5), (0, 9), (0, 10), (0, 14), (1, 2), (1, 4), (1, 7), (1, 14), (2, 3), (2, 4), (2, 5), (2, 7), (2, 8), (2, 9), (2, 15), (3, 0), (3, 3), (3, 13), (3, 14), (3, 15), (4, 2), (4, 6), (4, 8), (4, 10), (4, 11), (5, 8), (6, 14), (6, 15), (7, 2), (7, 11), (7, 13), (8, 0), (9, 3), (9, 4), (9, 5), (9, 6), (9, 12), (9, 15), (10, 1), (10, 4), (10, 9), (10, 10), (10, 13), (10, 14), (11, 1), (11, 4), (11, 5), (11, 8), (12, 1), (12, 4), (12, 12), (12, 13), (12, 15), (13, 5), (13, 12), (13, 13), (14, 5), (14, 14), (15, 5), (15, 7), (15, 11), (15, 12), (15, 15), (16, 6), (16, 7), (16, 9), (16, 11), (16, 13), (16, 14), (17, 0), (17, 1), (17, 12), (18, 3), (18, 11), (18, 13), (19, 1), (19, 4), (20, 0), (20, 1), (20, 14), (21, 0), (21, 2), (21, 7)]
len(circ_246)

86

In [ ]:
sorted(list(set(circ_123)-set(circ_246)))

[(1, 11),
 (2, 10),
 (2, 14),
 (3, 4),
 (3, 9),
 (4, 7),
 (4, 9),
 (4, 13),
 (5, 5),
 (5, 11),
 (5, 12),
 (5, 13),
 (6, 12),
 (8, 10),
 (11, 3),
 (11, 6),
 (15, 10)]

In [ ]:
sorted(list(set(circ_246)-set(circ_123)))

[(0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 10),
 (0, 14),
 (1, 4),
 (1, 7),
 (2, 8),
 (2, 15),
 (3, 14),
 (4, 2),
 (4, 10),
 (4, 11),
 (5, 8),
 (6, 14),
 (7, 11),
 (7, 13),
 (8, 0),
 (9, 3),
 (9, 4),
 (9, 6),
 (9, 12),
 (9, 15),
 (10, 9),
 (10, 10),
 (10, 13),
 (11, 8),
 (12, 1),
 (12, 4),
 (12, 12),
 (12, 13),
 (13, 5),
 (13, 12),
 (15, 5),
 (15, 11),
 (15, 12),
 (15, 15),
 (16, 6),
 (16, 7),
 (16, 9),
 (16, 11),
 (16, 13),
 (16, 14),
 (17, 0),
 (17, 1),
 (18, 3),
 (18, 11),
 (18, 13),
 (19, 4),
 (20, 0),
 (20, 1),
 (20, 14),
 (21, 0),
 (21, 2),
 (21, 7)]